# Collaborative Filtering

In [21]:
import os
import sys

sys.path.append("/Users/eric/Code/notflix/src")

import csv
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.decomposition.truncated_svd import TruncatedSVD
from sklearn.neighbors import NearestNeighbors

from src.data.db import common, utils, DB_HOST

### SVD on Items

In [6]:
ratings = pd.read_csv("../../datasets/movielens/ml-20m/ratings.csv")
ratings["like"] = ratings["rating"].apply(lambda r: 1 if r > 3 else 0)

In [7]:
def sparse_matrix_from_df(df, groupby, indicator):
    rows_u = list(df.index.unique())
    columns_u = list(df[groupby].unique())

    data = df[indicator].tolist()
    
    row = pd.Series(df.index)\
            .astype('category', categories=rows_u)\
            .cat.codes
    col = df[groupby]\
            .astype('category', categories=columns_u)\
            .cat.codes
            
    sparse_matrix = csr_matrix((data, (row, col)), 
                               shape=(len(rows_u), len(columns_u)))

    return sparse_matrix, rows_u, columns_u

In [25]:
matrix, items, users = sparse_matrix_from_df(ratings.sample(500000).set_index('movieId'), 
                                             "userId",
                                             "like")

/Users/eric/Code/virtualenvs/data3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys
/Users/eric/Code/virtualenvs/data3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  


In [26]:
svd = TruncatedSVD(n_components=150, n_iter=3)
%time lower_rank_matrix = svd.fit_transform(matrix)

CPU times: user 19.9 s, sys: 1.99 s, total: 21.9 s
Wall time: 11.9 s


In [27]:
knn = NearestNeighbors(20, metric="cosine")
knn.fit(lower_rank_matrix)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=20, p=2, radius=1.0)

In [35]:
to_insert = []

for i, movie_id in enumerate(items):
    metrics, neighbors = knn.kneighbors(lower_rank_matrix[i].reshape(1, -1))
    
    for metric, item_id in zip(metrics[0], map(lambda n: items[n], neighbors[0])):
        r = common.Recommendation(**{"engine_name": "SVDItems",
                                     "source_product_id": movie_id,
                                     "recommended_product_id": item_id,
                                     "score": 1-metric})
        
        to_insert.append(r)